In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import ast
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
def parse_dict_string(value):
    """
    Esta función comprueba si un valor es nulo, de lo contrario, utiliza la función literal_eval 
    para evaluar una cadena como un diccionario de Python.
    
    """
    if pd.isna(value):
        return None
    else: 
        return ast.literal_eval(value)


def convert(text):
    """ 
    Esta función comprueba si un valor es nulo, de lo contrario, realiza un ciclo for 
    para añadir los nombres en una lista a partir de como un diccionario de Python.

    """
    L = []
    if text == None:
        pass
    else:
        for i in ast.literal_eval(text):
            L.append(i['name']) 
        return L 


def fetch_director(text):
    """
    Esta función realiza un ciclo for para añadir los nombres, solo si su trabajo es de director, 
    en una lista a partir de como un diccionario de Python.

    """
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 

In [3]:
# Se carga el dataset movies_dataset.csv a un dataframe en pandas.
df_movies = pd.read_csv('movies_dataset.csv')

C:\Users\Miguel\AppData\Local\Temp\ipykernel_12128\1969850628.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('movies_dataset.csv')


In [4]:
# Se carga el dataset credits.csv a un dataframe en pandas.
df_credits = pd.read_csv('credits.csv')

# Se le aplica la función convert a la columna cast.
df_credits['cast'] = df_credits['cast'].apply(convert)

# Se le aplica la función fetch_director a la columna crew.
df_credits['crew'] = df_credits['crew'].apply(fetch_director)

# Se renombra la columna crew a "director".
df_credits.rename(columns={'crew': 'director'}, inplace=True)

In [5]:
# Se convierte la columna id a numérico.
df_movies['id'] = pd.to_numeric(df_movies['id'], errors='coerce')

# Se eliminan los valores nulos de la columna id.
df_movies = df_movies.dropna(subset=['id'])

# Se convierte el tipo de los valores a int64.
df_movies['id'] = df_movies['id'].astype('int64')

# Se realiza el merge con el dataframe df_credits.
df_movies = df_movies.merge(df_credits, how='inner', on='id')

In [6]:
# Se le aplica la función parse_dict_string a la columna "belongs_to_collection".
df_movies['belongs_to_collection'] = df_movies['belongs_to_collection'].apply(parse_dict_string)

# Se normaliza la columna y se guarda en la variable "collection".
collection = pd.json_normalize(df_movies['belongs_to_collection']).fillna('')

# Se renombran las columnas id y name.
collection = collection.rename(columns= {'name': 'name_collection'})

# Se eliminan las columnas que no nos aportan ningún valor.
collection = collection['name_collection']

# Se concatenan los 2 dataframes por medio del indice.
df_movies = pd.concat([df_movies, collection], axis=1)

# Finalmente, se elimina la columna "belongs_to_collection"
df_movies.drop(columns='belongs_to_collection', inplace=True)

In [7]:
# Se aplica la función convert a la columna genres.
df_movies['genres'] = df_movies['genres'].apply(convert)


In [8]:
# Se eliminan los datos nulos de la columna production_companies.
df_movies = df_movies.dropna(subset=['production_companies'])

# Se aplica la función convert a la columna production_companies, production_countries y spoken_languages.
df_movies['production_companies'] = df_movies['production_companies'].apply(convert)
df_movies['production_countries'] = df_movies['production_countries'].apply(convert)
df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(convert)

In [9]:
# Se eliminan los valores nulos de la columna release_date.
df_movies = df_movies.dropna(subset=['release_date'])

# Se cambian el tipo de dato a datetime de la columna release_date.
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'])

# Se crea una nueva columna "release_year" con el año de estreno de cada pelicula a partir de la colu,na release_date.
df_movies['release_year'] = df_movies['release_date'].dt.year

In [10]:
# Se convierten el tipo de datos de las columnas revenue y budget a int64.
df_movies['revenue'] = df_movies['revenue'].astype('int64')
df_movies['budget'] = df_movies['budget'].astype('int64')

In [11]:
# Se crea una nueva columna "return" a partir de la división entre las columnas revenue y budget, imputando el valor 0 sobre los datos nulos.
df_movies['return'] = round(df_movies['revenue'].div(df_movies['budget']), 2).fillna(0)

# Se reemplaza los datos "inf" por valor 0 en la columna return.
df_movies['return'] = np.where(np.isinf(df_movies['return']), 0, df_movies['return'])

In [12]:
# Se convierte el tipo de dato de la columna popularity a float64.
df_movies['popularity'] = df_movies['popularity'].astype('float64')

# Se ordena el dataframe por score de popularidad de mayor a menor.
df_movies.sort_values(by='popularity', ascending=False ,inplace=True)

# Se eliminan duplicados.
df_movies.drop_duplicates(subset=['id'], keep='first', inplace=True)

In [13]:
# Se eliminan las columnas que no serán utilizadas en este proyecto.
df_movies = df_movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'])

In [15]:
df_movies.head(3)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,director,name_collection,release_year,return
30780,74000000,"[Family, Animation, Adventure, Comedy]",211672,en,"Minions Stuart, Kevin and Bob are recruited by...",547.488298,"[Universal Pictures, Illumination Entertainment]",[United States of America],2015-06-17,1156730962,91.0,[English],Released,"Before Gru, they had a history of bad bosses",Minions,6.4,4729.0,"[Sandra Bullock, Jon Hamm, Michael Keaton, All...","[Kyle Balda, Pierre Coffin]",Despicable Me Collection,2015,15.63
33439,149000000,"[Action, Adventure, Fantasy]",297762,en,An Amazon princess comes to the world of Man t...,294.337037,"[Dune Entertainment, Atlas Entertainment, Warn...",[United States of America],2017-05-30,820580447,141.0,"[Deutsch, English]",Released,Power. Grace. Wisdom. Wonder.,Wonder Woman,7.2,5025.0,"[Gal Gadot, Chris Pine, Robin Wright, Danny Hu...",[Patty Jenkins],Wonder Woman Collection,2017,5.51
42298,160000000,"[Family, Fantasy, Romance]",321612,en,A live-action adaptation of Disney's version o...,287.253654,"[Walt Disney Pictures, Mandeville Films]","[United Kingdom, United States of America]",2017-03-16,1262886337,129.0,[English],Released,Be our guest.,Beauty and the Beast,6.8,5530.0,"[Emma Watson, Dan Stevens, Luke Evans, Kevin K...",[Bill Condon],,2017,7.89


In [14]:
# Se importa a un nuevo archivo csv.
df_movies.to_csv('movies_clean.csv', index=False)